In [3]:
# !pip install pycandela # https://candela.readthedocs.io/en/latest/
# !pip install networkx

In [4]:
from demopy.notebook_imports import *
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.set_option("display.max_rows", 999)
pd.get_option("display.max_rows")

999

In [13]:
import ipywidgets as widgets
from traitlets import Unicode, validate, Dict, List
import json

class CytoscapeWidget(widgets.DOMWidget):
    _view_name = Unicode('CytoscapeView').tag(sync=True) # This is the name of the Javascript View
    _view_module = Unicode('cytoscapewidget').tag(sync=True) # This is the name of the defined java script method
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = Unicode('Hello World!').tag(sync=True)
    domid = Unicode('cy').tag(sync=True)
    elements = Unicode('{}').tag(sync=True)
    sytleFile = Unicode('https://js.cytoscape.org/demos/labels/cy-style.json').tag(sync=True)

In [15]:
def update_and_return(d, ud):
    nd = {}
    nd.update(d)
    nd.update(ud)
    return nd

def pop_data_in_nodes(graph_repr):
    new_graph_repr = {}
    new_graph_repr.update(graph_repr)
    new_graph_repr["nodes"] =  [update_and_return(x, x["data"].pop('popme') if 'popme' in x["data"] else {}) for x in new_graph_repr["nodes"]]
    return new_graph_repr

def pop_data_in_edges(graph_repr):
    new_graph_repr = {}
    new_graph_repr.update(graph_repr)
    new_graph_repr["edges"] =  [update_and_return(x, x["data"].pop('popme') if 'popme' in x["data"] else {}) for x in new_graph_repr["edges"]]
    return new_graph_repr

In [16]:
import networkx as nx
from uuid import uuid4


# Set the id of the current element
ID = str(uuid4())
set_id_for_dom_element_of_output_for_current_cell(ID)

# Make a graph with networkx
G = nx.Graph()
G.add_node(1, label="1", popme={"classes": "bottom-center profile"})
G.add_node(2, label="2", popme={"classes": "bottom-center"})
G.add_node(3, label="3", popme={"classes": "bottom-center"})
# for x in range(3,100):
#     G.add_node(x, label="{}".format(x), popme={"classes": "bottom-center"})
G.add_edge(1, 2, label="e1", popme={"classes": "autorotate"})
G.add_edge(1, 3, label="e2", popme={"classes": "autorotate"})
G.add_edge(1, 4, label="e3", popme={"classes": "autorotate"})
G.add_edge(1, 5, label="e4", popme={"classes": "autorotate"})

x = CytoscapeWidget()
x.elements = json.dumps(
    pop_data_in_nodes(
        pop_data_in_edges(
            nx.readwrite.cytoscape_data(G)["elements"]
        )
    )
)
x.domid = ID

# Use this as the starting spot of figuring out what to plot in the profile ...

display(x)

<IPython.core.display.Javascript object>

CytoscapeWidget(domid='ce35a93e-fc1e-45e1-bcf6-fd1a13caa2a8', elements='{"nodes": [{"data": {"label": "1", "id…

In [1]:
%%javascript

require.undef('cytoscapewidget');

require.config({
  paths: {
    'cytoscape': 'https://unpkg.com/cytoscape@3.2.16/dist/cytoscape.min',
    'popper': 'https://unpkg.com/popper.js@1.14.4/dist/umd/popper.min',
    'cytoscape-popper': 'https://unpkg.com/cytoscape-popper@1.0.2/cytoscape-popper',
    'tippy': 'https://unpkg.com/tippy.js@2.0.9/dist/tippy.all'
  },
  shim: {
    'cytoscape': {
        exports: 'cytoscape'
    },
    "popper": {
        deps: ['jquery'],
        exports: "Popper"
    },
    'cytoscape-popper': ['cytoscape', 'popper.js']
    }
  },
  waitSeconds: 30
});

define('cytoscapewidget', ["@jupyter-widgets/base", "cytoscape", "cytoscape-popper"], function(widgets, cytoscape, popper) {
//     popper( cytoscape ); // Register popper
    
    var CytoscapeView = widgets.DOMWidgetView.extend({
        _render: function() {
            // Python sends over to the JavaScript Json STRINGS! They dont actually get decoded into the json types ...
            var elements = JSON.parse(this.model.get('elements'));
            var elementId = this.model.get('domid')
            // We have to use .get on model because its an attribute (model.attriubte.styleFile). It looks like .get knows how to look in the attributes ...
            fetch(this.model.get('sytleFile'))
            .then(function(res) {
                return res.json()
            })
            .then(function(style) {
                var fullStyle = style.slice(2,).concat([
                    {
                        selector: 'node',
                        style: {
                            'height': '128',
                            'width': '128',
                            'label': 'data(label)'
                        }
                    },
                    {   
                        selector: 'edge',
                        style: {
                            'label': 'data(label)',
                            'curve-style': 'haystack',
                            'haystack-radius': 0,
                            'width': '32',
                            'opacity': 0.5,
                            'line-color': '#a8eae5'
                        }
                    },
                    {
                        selector: '.profile',
                        style: {
                            'background-image': 'https://cdn3.iconfinder.com/data/icons/vector-icons-6/96/256-128.png',
//                             'background-size': '128 128'
                        }
                    }
                ]);
                
//              var elementToDraw = element.get(0); elementToDraw.id = "cy";
                var elementToDraw = document.getElementById(elementId);
                elementToDraw.style.height = "900px";
                elementToDraw.style.width = "1200px";
                var cy = cytoscape({
                  container: elementToDraw,
                  style: fullStyle,
                  boxSelectionEnabled: false,
                  autounselectify: true,
                  layout: {
                    name: 'concentric',
                    concentric: function( node ){
                      console.log(node);
                      // there is node.data ...
                      return node.degree();
                    },
                    levelWidth: function( nodes ){
                      return 2;
                    },
                    equidistant: true,
                    fit: true, // whether to fit the viewport to the graph
                    spacingFactor: 3
                  },
                  elements: elements,
//                   zoomingEnabled: false,
                  zoom: -1
                });
                
//                 let node = cy.nodes().first();
//                 let ref = node.popperRef(); // used only for positioning
//                 let tippy = new Tippy(ref, { // tippy options:
//                   html: (() => {
//                     let content = document.createElement('div');
//                     content.innerHTML = 'Tippy content';
//                     return content;
//                   })(),
//                   trigger: 'manual' // probably want manual mode
//                 }).tooltips[0];

//                 node.on('tap', () => tippy.show());

//                 // you can use qtip's regular options
//                 // see http://qtip2.com/
//                 cy.$('#1').qtip({
//                   content: 'Hello!',
//                   position: {
//                     my: 'top center',
//                     at: 'bottom center'
//                   },
//                   style: {
//                     classes: 'qtip-bootstrap',
//                     tip: {
//                       width: 16,
//                       height: 8
//                     }
//                   }
//                 });
                
            });
        },
        render: function() {
            this._render();
            this.model.on('change:elements', this._render, this);
            this.model.on('change:domid', this._render, this);
        }
    });

    return {
        CytoscapeView : CytoscapeView
    };
});

<IPython.core.display.Javascript object>

In [ ]:
import ipywidgets as widgets
from traitlets import Unicode, validate


class CytoscapeWidget(widgets.DOMWidget):
    _view_name = Unicode('CytoscapeView').tag(sync=True)
    _view_module = Unicode('cytoscapewidget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = Unicode('Hello World!').tag(sync=True)


In [ ]:
%%javascript

// 'cytoscape': 'https://js.cytoscape.org/js/cytoscape.min.js'
// 'cytoscape': 'https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.2.16/cytoscape'
// 'https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.5.1/cytoscape.min',

require.config({
  paths: {
    'cytoscape': 'https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.2.15/cytoscape.min'
  },
  shim: {
    'cytoscape': {
        exports: 'cytoscape'
    }
  },
  waitSeconds: 30
});

In [ ]:
%%javascript
require.undef('cytoscapewidget');

// define('cytoscapewidget', ["@jupyter-widgets/base"], function(widgets) {
define('cytoscapewidget', ["@jupyter-widgets/base", "cytoscape"], function(widgets, cytoscape) {
    var CytoscapeView = widgets.DOMWidgetView.extend({
        render: function() {
            fetch('https://js.cytoscape.org/demos/labels/cy-style.json')
            .then(function(res) {
                return res.json()
            })
            .then(function(style) {
                element.get(0).id = "cy";
                var cy = cytoscape({
                  container: element.get(0),
                  style: style,
                  elements: [
                    { data: { label: 'top left' }, classes: 'top-left' },
                    { data: { label: 'top center' }, classes: 'top-center' },
                    { data: { label: 'top right' }, classes: 'top-right' },
                    { data: { label: 'center left' }, classes: 'center-left' },
                    { data: { label: 'center center' }, classes: 'center-center' },
                    { data: { label: 'center right' }, classes: 'center-right' },
                    { data: { label: 'bottom left' }, classes: 'bottom-left' },
                    { data: { label: 'bottom center' }, classes: 'bottom-center' },
                    { data: { label: 'bottom right' }, classes: 'bottom-right' },
                    { data: { label: 'multiline auto foo bar baz' }, classes: 'multiline-auto' },
                    { data: { label: 'outline' }, classes: 'outline' },
                    { data: { id: 'ar-src' } },
                    { data: { id: 'ar-tgt' } },
                    { data: { source: 'ar-src', target: 'ar-tgt', label: 'autorotate (move my nodes)' }, classes: 'autorotate' },
                    { data: { label: 'background' }, classes: 'background' }
                  ]
                });
            });
        }
//        render: function() {
//             this.value_changed();
//             this.model.on('change:value', this.value_changed, this);
//         },

//         value_changed: function() {
//             this.el.textContent = this.model.get('value');
//         },
    });

    return {
        CytoscapeView : CytoscapeView
    };
});

In [ ]:
CytoscapeWidget()

In [ ]:
x.value = "asdf"

In [ ]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from([1, 2, 3])
G.add_edges_from([(1,2)])
nx.readwrite.cytoscape_data(G)

In [ ]:
from py2cytoscape.cytoscapejs import render
render(nx.readwrite.cytoscape_data(G))